In [ ]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas_datareader import data
from sklearn.preprocessing import MinMaxScaler
import pandas.tseries.offsets as offsets

from tqdm import tqdm

In [ ]:
symbols_names = {
    '6501.JP': 'Hitachi',
    '6502.JP': 'Toshiba',
    '4776.JP': 'Cybozu',
    '6027.JP': 'Bengo4.com',
    '4385.JP': 'Mercari',
}

In [ ]:
# 関数を定義

stock_data = {}
error_symbols = []

y = {}
scaler = {}

train_window_size = 7

test_size = 30
extending_seq = {}

epochs = 20

predicted_nomalized_labels_list = {}
predicted_nomalized_labels_array_1d = {}
predicted_nomalized_labels_array_2d = {}
predicted_labels_array_2d = {}

real_last_date_timestamp = {}

future_first_date_timestamp = {}
future_first_date_series_object = {}
future_first_date_str = {}

future_last_date_timestamp = {}
future_last_date_series_object = {}
future_last_date_str = {}

furture_period = {}

plot_start_date_timestamp = {}

In [ ]:
def input_data(seq, ws):
    out = []
    L = len(seq)

    for i in range(L-ws):
        window = seq[i:i+ws]
        label = seq[i+ws:i+ws+1]
        out.append((window, label))

    return out

In [ ]:
class Model(nn.Module):

    def __init__(self, input=1, h=50, output=1):
        super().__init__()
        self.hidden_size = h

        self.lstm = nn.LSTM(input, h)
        self.fc = nn.Linear(h, output)

        self.hidden = (
            torch.zeros(1, 1, h),
            torch.zeros(1, 1, h)
        )

    def forword(self, seq):

        out, _ = self.lstm(
            seq.view(len(seq), 1, -1),
            self.hidden
        )

        out = self.fc(
            out.view(len(seq), -1)
        )

        return out[-1]

In [ ]:
for j, key in enumerate(tqdm(symbols_names)): # tqdmあり



    try:
        stock_data[j] = data.DataReader(key, 'stooq').sort_values('Date', ascending=True)
        stock_data[j] = stock_data[j].drop(['Open', 'High', 'Low', 'Volume'], axis=1)
    except:
        error_symbols.append(key)



    y[j] = stock_data[j]['Close'].values

    scaler[j] = MinMaxScaler(feature_range=(-1, 1))
    scaler[j].fit(y[j].reshape(-1, 1))
    y[j] = scaler[j].transform(y[j].reshape(-1, 1))

    y[j] = torch.FloatTensor(y[j]).view(-1)



    train_data = input_data(y[j], train_window_size)

    torch.manual_seed(123)
    model = Model()
    criterion = nn.MSELoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01)


    for epoch in range(epochs):
        

        for train_window, correct_label in train_data:

            optimizer.zero_grad()

            model.hidden = (
                torch.zeros(1, 1, model.hidden_size),
                torch.zeros(1, 1, model.hidden_size)
            )

            train_predicted_label = model.forword(train_window)
            train_loss = criterion(train_predicted_label, correct_label)

            train_loss.backward()
            optimizer.step()


        extending_seq[j] = y[j][-test_size:].tolist()


        for i in range(test_size):

            test_window = torch.FloatTensor(extending_seq[j][-test_size:])

            with torch.no_grad():

                model.hidden = (
                    torch.zeros(1, 1, model.hidden_size),
                    torch.zeros(1, 1, model.hidden_size)
                )

                test_predicted_label = model.forword(test_window)
                extending_seq[j].append(test_predicted_label.item())



    predicted_nomalized_labels_list[j] = extending_seq[j][-test_size:]
    predicted_nomalized_labels_array_1d[j] = np.array(predicted_nomalized_labels_list[j])
    predicted_nomalized_labels_array_2d[j] = predicted_nomalized_labels_array_1d[j].reshape(-1, 1)
    predicted_labels_array_2d[j] = scaler[j].inverse_transform(predicted_nomalized_labels_array_2d[j])



    real_last_date_timestamp[j] = stock_data[j].index[-1]

    future_first_date_timestamp[j] = real_last_date_timestamp[j] + offsets.Day()
    future_first_date_series_object[j] = pd.Series(future_first_date_timestamp[j]).astype(str)
    future_first_date_str[j] = future_first_date_series_object[j][0]

    future_last_date_timestamp[j] = future_first_date_timestamp[j] + offsets.Day(30)
    future_last_date_series_object[j] = pd.Series(future_last_date_timestamp[j]).astype(str)
    future_last_date_str[j] = future_last_date_series_object[j][0]

    furture_period[j] = np.arange(future_first_date_str[j], future_last_date_str[j], dtype='datetime64')

    plot_start_date_timestamp[j] = real_last_date_timestamp[j] + offsets.Day(-90)


    fig = plt.figure(figsize=(12, 4))
    plt.title('Stock Price Prediction')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.grid(True)

    plt.plot(stock_data[j]["Close"][plot_start_date_timestamp[j]:], label=symbols_names.get(key))

    plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left", borderaxespad=0, fontsize=18)

    plt.plot(furture_period[j], predicted_labels_array_2d[j])

    plt.show()